In [ ]:
import importlib
import myutils
importlib.reload(myutils)

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
import pandas as pd

In [ ]:
df = pd.read_csv("csv_files/fangraphs_leaderboard_24.csv")
df["vFA (pi)"].fillna(df["vFA (pi)"].mean(), inplace=True)
print(df["vFA (pi)"].median())

In [ ]:
df2 = pd.read_csv("csv_files/fangraphs-leaderboards 2008.csv")
df2["vFA (pi)"].fillna(df2["vFA (pi)"].mean(), inplace=True)
print(df2["vFA (pi)"].median())

In [ ]:
df3 = pd.read_csv("csv_files/2008-velo.csv")
df3["FB%"].fillna(df3["FB%"].mean(), inplace=True)
print(df3["FB%"].median())

In [ ]:
df4 = pd.read_csv("csv_files/fangraphs_leaderboard_24.csv")
df4["FBv"].fillna(df4["FBv"].mean(), inplace=True)
print(df4["FBv"].median())

In [ ]:
# get rid of same named columns 
df_clean = df[["Name"]]
df = df.drop(columns=['Team', 'NameASCII', 'PlayerId', 'MLBAMID'], errors="ignore")
df4["Name"] = df4["Name"].astype(str)
df["Name"] = df["Name"].astype(str)
new_df = df4.join(df_clean.set_index("Name"), on="Name", how='left')
print(new_df)
len(df4)

In [ ]:
injury_page = pd.read_csv("csv_files/injury_report_24.csv")
pitch_breakdown = pd.read_csv("csv_files/fangraphs_leaderboard_all_pitchers.csv")

counter = len(injury_page[injury_page["Pos"] == "SP"])
print(counter)

855 players pitched during the 2024 season. 229 starting pitchers were listed as injured during the 2024 season. 199 of them played in a game. 

In [ ]:
injured_pitchers = injury_page[injury_page["Pos"] == "SP"]
injured_pitchers_with_age = pd.merge(injured_pitchers, pitch_breakdown, on="Name", how="inner")

Here is the breakdown by age:

In [ ]:
age_counts = injured_pitchers_with_age["Age"].value_counts().sort_index()  # Sort by age
print(age_counts)

In [ ]:
injured_pitchers_with_age["vFA (pi)"].fillna(injured_pitchers_with_age["vFA (pi)"].mean(), inplace=True)
print(injured_pitchers_with_age["vFA (pi)"].median())

print(injured_pitchers_with_age["IP"].median()/injured_pitchers_with_age["G"].median())

In [ ]:
merged = pd.merge(
    pitch_breakdown,
    injured_pitchers,
    on="Name",        
    how="left",       # keep all pitchers
    indicator=True    # adds a "_merge" column
)

# add classification of 1 if injured, 0 if not
pitch_breakdown["injury_label"] = merged["_merge"].apply(lambda x: 1 if x == "both" else 0)


In [ ]:
features = [
    'Age',
    'IP',
    'vFA (pi)',
    'FB%',
    'SL%',
    'CH%',
]

X = pitch_breakdown[features]
y = pitch_breakdown["injury_label"] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('model', LogisticRegression(max_iter=1000))  # use more iterations if it doesn't converge
])

# Fit the pipeline
pipeline.fit(X_train, y_train)
